# 多項式環における計算

googleで検索して見つけた以下のページの行列の多項式への作用の定義を実装

https://ask.sagemath.org/question/60980/how-to-define-group-actions-on-sagemath/

In [1]:
from sage.categories.action import Action
from sage.groups.matrix_gps.matrix_group import is_MatrixGroup
from sage.rings.polynomial.multi_polynomial_ring import is_MPolynomialRing

class InverseLinearMapSubstitutionAction(Action):
    def __init__(self, G, S):
        if not is_MatrixGroup(G):
            raise TypeError("Not a matrix group: %s" % G)
        if not is_MPolynomialRing(S):
            raise TypeError("Not a multivariate polynomial ring: %s" % S)
        if not G.degree() == S.ngens():
            raise ValueError("Degree of matrix group (%d) does not match number of generators of polynomial ring (%d)" % (G.degree(), S.ngens()))
        if not S.base_ring().has_coerce_map_from(G.base_ring()):
            raise ValueError("Base rings not compatible")
        super().__init__(G, S, is_left=True, op=operator.mul)

    def _act_(self, g, f):
        return f(*(g.inverse() * vector(self.domain(), self.domain().gens())))

    def _repr_name_(self):
        return "inverse linear map substitution action"

剰余類の代表系を計算する関数を実装（重めの処理になっている..）

In [2]:
from sage.groups.group import is_Group


def cosetRep(G,H):
    if not is_Group(G):
        raise TypeError("Not a group: %s" % G)
    if not is_Group(H):
        raise TypeError("Not a group: %s" % H)
    
    rep=[]
    tmp=set()
    coset=set()
    flag=0
    g=G.list()
    h=H.list()
    
    for i in range(len(g)):
        for j in range(len(h)):
            p=g[i]*h[j]
            if p in tmp:
                flag=1
                break;
            else:
                coset.add(p)
        if flag==0:
            rep.append(g[i])
            tmp = tmp.union(coset)
        coset.clear()
        flag=0
        if len(rep)*len(h)==len(g):
            break;
    tmp.clear()
    return rep

パラメタを設定

In [3]:
n=7
r=2

A型のWeyl group $W_G$を生成

In [4]:
Delta_G=RootSystem(['A',n-1])
W_G=WeylGroup(Delta_G)

$W_G$のpositive rootとそれが定めるreflectionの組を格納

In [5]:
s=W_G.reflections()

$W_G$のpositive rootを取得

In [6]:
P_G=list(s.keys());P_G

[(1, -1, 0, 0, 0, 0, 0),
 (0, 1, -1, 0, 0, 0, 0),
 (0, 0, 1, -1, 0, 0, 0),
 (0, 0, 0, 1, -1, 0, 0),
 (0, 0, 0, 0, 1, -1, 0),
 (0, 0, 0, 0, 0, 1, -1),
 (1, 0, -1, 0, 0, 0, 0),
 (0, 1, 0, -1, 0, 0, 0),
 (0, 0, 1, 0, -1, 0, 0),
 (0, 0, 0, 1, 0, -1, 0),
 (0, 0, 0, 0, 1, 0, -1),
 (1, 0, 0, -1, 0, 0, 0),
 (0, 1, 0, 0, -1, 0, 0),
 (0, 0, 1, 0, 0, -1, 0),
 (0, 0, 0, 1, 0, 0, -1),
 (1, 0, 0, 0, -1, 0, 0),
 (0, 1, 0, 0, 0, -1, 0),
 (0, 0, 1, 0, 0, 0, -1),
 (1, 0, 0, 0, 0, -1, 0),
 (0, 1, 0, 0, 0, 0, -1),
 (1, 0, 0, 0, 0, 0, -1)]

$W_H$を生成するreflectionを定めるrootの集合$P_H$を用意する.

In [7]:
#Delta_Hのpositive rootを格納するlistを用意
P_H=[]

#Delta_Gのpositive rootのうちDelta_Hのpositive rootになるものを抽出
for i in range(len(P_G)):
    #最初のr個の成分がすべて0か, 最後のn-r個の成分がすべて0の場合にDelta_Hのpositive rootであると判断
    
    if [ P_G[i][j] for j in range(0,r) ]==[ 0 for k in range(0,r) ] or [ P_G[i][j] for j in range(r,n) ]==[ 0 for k in range(r,n) ]:
        P_H.append(P_G[i])
P_H

[(1, -1, 0, 0, 0, 0, 0),
 (0, 0, 1, -1, 0, 0, 0),
 (0, 0, 0, 1, -1, 0, 0),
 (0, 0, 0, 0, 1, -1, 0),
 (0, 0, 0, 0, 0, 1, -1),
 (0, 0, 1, 0, -1, 0, 0),
 (0, 0, 0, 1, 0, -1, 0),
 (0, 0, 0, 0, 1, 0, -1),
 (0, 0, 1, 0, 0, -1, 0),
 (0, 0, 0, 1, 0, 0, -1),
 (0, 0, 1, 0, 0, 0, -1)]

Delta_Hのpositive rootが定めるreflectionにより生成される$W_G$の部分群$W_H$を定義

In [8]:
gen=[s[P_H[i]] for i in range(len(P_H)) ]
W_H=W_G.subgroup(gen)

多項式環とWeyl group $W_G$ の多項式環への作用を生成

In [9]:
R = PolynomialRing(QQ, 'x', W_G.degree())
a = InverseLinearMapSubstitutionAction(W_G, R); a

Left inverse linear map substitution action by Weyl Group of type ['A', 6] (as a matrix group acting on the ambient space) on Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

変数のリストを用意

In [10]:
x=R.gens();x

(x0, x1, x2, x3, x4, x5, x6)

$ W_H $の作用で不変となる多項式を用意

In [11]:
f=(sum(x[l] for l in range(0,r)))^(r*(n-r))
factor(f)

(x0 + x1)^10

冒頭で実装した関数を用いて, $W_G/W_H$の代表系を計算

In [12]:
rep=cosetRep(W_G,W_H)

rootに対応する多項式を用意

In [13]:
Roots=set(P_G)-set(P_H)
alpha=[ -sum( r[l]*x[l] for l in range(len(x))) for r in Roots ];alpha

[-x1 + x6,
 -x0 + x6,
 -x0 + x5,
 -x1 + x5,
 -x0 + x2,
 -x1 + x2,
 -x1 + x3,
 -x0 + x3,
 -x0 + x4,
 -x1 + x4]

目的の計算を実施

In [14]:
sum( [ a(rep[i],f)/a( rep[i] , prod( alpha ) ) for i in range(len(rep)) ])

42

数値計算は未実装...

In [15]:
#前の固定化群として$W_H$を定義していた部分

#$W_H$を直接定義したいが, うまい方法を思いつかないため, ここでは$f$の固定化部分群として$f$を固定する元を求めて逆に定義（処理が重い..）
#stab=[]
#g=G.list()
#for i in range(len(g)):
#    if a(g[i].to_matrix(),f)==f:
#        stab.append(g[i])
#W_H=W_G.subgroup(stab)

--------以降は以前のノートの内容

パラメタ設定
（今はrは2で固定）

In [16]:
n=3
r=2

多項式環を用意

In [17]:
R0 = PolynomialRing(QQ, 'x', n)
R1= PolynomialRing(R0,'y',2)
phi = R1.flattening_morphism()
R = phi.codomain()

変数のリストを用意

In [18]:
x=R.gens();x

(x0, x1, x2, y0, y1)

$f=(y_0+y_1)^{r(n-r)}$を定義

In [19]:
f=(x[n]+x[n+1])^(r*(n-r))
factor(f)

(y0 + y1)^2

In [20]:
sum(sum( f.subs({x[n]:x[i],x[n+1]:x[j]})/prod([(x[k]-x[i])*(x[k]-x[j])
    for k in set(range(n))-{i,j}])
    for j in range(i+1,n))
    for i in range(0,n-1))

1

$\operatorname{deg} \operatorname{Gr}(2,n) = \frac{(2 n - 4)!}{(n - 2)! (n - 1)!}$と比較

In [21]:
factorial(2*n - 4)/(factorial(n - 2)*factorial(n - 1)) 

1

# 数値計算

パラメタの設定
（今は$r$は2で固定）

In [22]:
n=20
r=2

乱数の長さ$n$のリストを用意

In [23]:
x = [RealField(1000)(random()) for i in range(n)];

$f(y_0,y_1)=(y_0+y_1)^{r(n-r)}$を定義

In [24]:
y = SR.var('y', r); f(y0,y1)=(y0+y1)^(r*(n-r)); f

(y0, y1) |--> (y0 + y1)^36

In [25]:
(sum(sum(f(x[i],x[j])/prod([(x[k]-x[i])*(x[k]-x[j])
    for k in set(range(n))-{i,j}])
    for j in range(i+1,n))
    for i in range(0,n-1))
).round()

477638700

$\operatorname{deg} \operatorname{Gr}(2,n) = \frac{(2 n - 4)!}{(n - 2)! (n - 1)!}$と比較

In [26]:
 factorial(2*n - 4)/(factorial(n - 2)*factorial(n - 1)) 

477638700